In [ ]:
!pip install pandas requests tqdm concurrent.futures


ERROR: Could not find a version that satisfies the requirement concurrent.futures (from versions: none)
ERROR: No matching distribution found for concurrent.futures


In [ ]:
import pandas as pd
import requests
import time

def get_uniprot_accessions(target_chembl_ids):
    uniprot_accessions = {}
    for target_id in target_chembl_ids:
        # Query ChEMBL API to get UniProt accessions
        url = f"https://www.ebi.ac.uk/chembl/api/data/target/{target_id}.json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            for component in data['target_components']:
                if 'accession' in component:
                    uniprot_accession = component['accession']
                    uniprot_accessions[target_id] = uniprot_accession
                    break  # Assuming one accession per target
        else:
            print(f"Failed to retrieve UniProt accession for {target_id}")
            uniprot_accessions[target_id] = "No UniProt accession found"
    return uniprot_accessions

def get_sequence(accession):
    url = f"https://rest.uniprot.org/uniprot/{accession}.fasta"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve sequence for accession {accession}")
        return "Sequence not available"

def extract_sequence(fasta_text):
    lines = fasta_text.splitlines()
    sequence = ''.join([line for line in lines if not line.startswith('>')])
    return sequence

def main(csv_file):
    # Load CSV file
    df = pd.read_csv(csv_file)

    # Extract Target ChEMBL IDs
    target_chembl_ids = df['target_chembl_id'].unique().tolist()

    # Retrieve UniProt accessions
    uniprot_accessions = get_uniprot_accessions(target_chembl_ids)

    # Create a dictionary mapping target IDs to sequences
    sequences = {}
    for target_id, accession in uniprot_accessions.items():
        if accession and accession != "No UniProt accession found":
            try:
                sequence_text = get_sequence(accession)
                sequence = extract_sequence(sequence_text)
                sequences[target_id] = sequence
            except Exception as e:
                print(f"Error fetching sequence for {target_id} with accession {accession}: {e}")
                sequences[target_id] = "Sequence not available"
        else:
            sequences[target_id] = "No UniProt accession or sequence available"

    # Add new columns to the DataFrame
    df['uniprot_accession'] = df['target_chembl_id'].map(uniprot_accessions)
    df['protein_sequence'] = df['target_chembl_id'].map(sequences)

    # Save the updated DataFrame to the CSV file
    df.to_csv('updated_compounds.csv', index=False)

if __name__ == "__main__":
    csv_file = "/content/cleaned_file.csv"
    main(csv_file)


Failed to retrieve sequence for accession ENSG00000284190
